In [51]:
# Import libraries
import pandas as pd
import numpy as np

In [52]:
# import data from filter_data
company_data = pd.read_excel("../output_sets/filter_data.xlsx")
company_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5850 entries, 0 to 5849
Data columns (total 81 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        5850 non-null   int64  
 1   Llave                             5850 non-null   int64  
 2   anio                              5850 non-null   int64  
 3   mes                               5850 non-null   int64  
 4   novedad                           5850 non-null   int64  
 5   id_estado                         5850 non-null   int64  
 6   imp_Imputacion                    5593 non-null   object 
 7   Corrección                        5610 non-null   object 
 8   id_numord                         5850 non-null   int64  
 9   nombre_estab                      5850 non-null   object 
 10  Dominio_39                        5850 non-null   int64  
 11  nombreDpto                        5850 non-null   object 
 12  nombre

In [53]:
# this lines gives format the numbers in this archive
pd.options.display.float_format = "{:,.0f}".format
company_nordem = 402

# 770450
# AJU_II_PA_PP_SUELD_EP


# pivot table of personal
def all_personal(company_data, company_nordem):
    company_data = company_data[company_data["id_numord"] == company_nordem]
    company_data = company_data.pivot_table(
        index=["anio", "mes"],
        values=[
            "id_numord",
            "II_PA_PP__NPERS_EP",
            "AJU_II_PA_PP_SUELD_EP",
            "II_PA_TD__NPERS_ET",
            "AJU_II_PA_TD_SUELD_ET",
            "II_PA_TI__NPERS_ETA",
            "AJU_II_PA_TI_SUELD_ETA",
            "II_PA_AP__AAEP",
            "AJU_II_PA_AP_AAS_AP",
            "II_PP_PP__NPERS_OP",
            "AJU_II_PP_PP_SUELD_OP",
            "II_PP_TD__NPERS_OT",
            "AJU_II_PP_TD_SUELD_OT",
            "II_PP_TI__NPERS_OTA",
            "AJU_II_PP_TI_SUELD_OTA",
            "II_PP_AP__APEP",
            "AJU_II_PP_AP_AAS_PP",
            "tot_empleo",
        ],
    )

    # averange salary per category
    company_averange_salary = company_data.copy()
    company_averange_salary["admin_per_dir"] = (
        company_averange_salary["AJU_II_PA_PP_SUELD_EP"]
        / company_averange_salary["II_PA_PP__NPERS_EP"]
    )
    company_averange_salary["admin_tem_dir"] = (
        company_averange_salary["AJU_II_PA_TD_SUELD_ET"]
        / company_averange_salary["II_PA_TD__NPERS_ET"]
    )
    company_averange_salary["admin_per_emp"] = (
        company_averange_salary["AJU_II_PA_TI_SUELD_ETA"]
        / company_averange_salary["II_PA_TI__NPERS_ETA"]
    )
    company_averange_salary["admin_per_apr"] = (
        company_averange_salary["AJU_II_PA_AP_AAS_AP"]
        / company_averange_salary["II_PA_AP__AAEP"]
    )
    company_averange_salary["pro_per_dir"] = (
        company_averange_salary["AJU_II_PP_PP_SUELD_OP"]
        / company_averange_salary["II_PP_PP__NPERS_OP"]
    )
    company_averange_salary["pro_tem_dir"] = (
        company_averange_salary["AJU_II_PP_TD_SUELD_OT"]
        / company_averange_salary["II_PP_TD__NPERS_OT"]
    )
    company_averange_salary["pro_per_emp"] = (
        company_averange_salary["AJU_II_PP_TI_SUELD_OTA"]
        / company_averange_salary["II_PP_TI__NPERS_OTA"]
    )
    company_averange_salary["pro_per_apr"] = (
        company_averange_salary["AJU_II_PP_AP_AAS_PP"]
        / company_averange_salary["II_PP_AP__APEP"]
    )

    """
        create table of administrative workers, 
        drop colums of the other categories, order columns and return this data
    """

    colums_to_delete_adm = [
        "AJU_II_PA_PP_SUELD_EP",
        "AJU_II_PA_TD_SUELD_ET",
        "AJU_II_PA_TI_SUELD_ETA",
        "AJU_II_PA_AP_AAS_AP",
        "AJU_II_PP_PP_SUELD_OP",
        "AJU_II_PP_TD_SUELD_OT",
        "AJU_II_PP_TI_SUELD_OTA",
        "AJU_II_PP_AP_AAS_PP",
        "tot_empleo",
        "II_PP_PP__NPERS_OP",
        "II_PP_TD__NPERS_OT",
        "II_PP_TI__NPERS_OTA",
        "II_PP_AP__APEP",
        "AJU_II_PP_AP_AAS_PP",
    ]

    company_admin_per = company_averange_salary.copy()
    company_admin_per = company_admin_per.drop(colums_to_delete_adm, axis=1)
    company_admin_per = company_admin_per.rename(
        columns={
            "II_PA_PP__NPERS_EP": "#_admin_per_dir",
            "II_PA_TD__NPERS_ET": "#_admin_tem_dir",
            "II_PA_TI__NPERS_ETA": "#_admin_tem_emp",
            "II_PA_AP__AAEP": "#_admin_ape",
        }
    )
    # order de columns and gives new names
    admin_order_var = [
        "#_admin_per_dir",
        "admin_per_dir",
        "#_admin_tem_dir",
        "admin_tem_dir",
        "#_admin_tem_emp",
        "admin_per_emp",
        "#_admin_ape",
        "admin_per_apr",
    ]
    # company admin personal table finished
    company_admin_per = company_admin_per.reindex(columns=admin_order_var)

    """ create table production workers,
    delete colums, order columns and create a table of production workers"""

    # colums to drop
    colums_to_delete_prd = [
        "II_PA_PP__NPERS_EP",
        "AJU_II_PA_PP_SUELD_EP",
        "II_PA_TD__NPERS_ET",
        "AJU_II_PA_TD_SUELD_ET",
        "II_PA_TI__NPERS_ETA",
        "AJU_II_PA_TI_SUELD_ETA",
        "II_PA_AP__AAEP",
        "AJU_II_PA_AP_AAS_AP",
        "AJU_II_PP_PP_SUELD_OP",
        "AJU_II_PP_TD_SUELD_OT",
        "AJU_II_PP_TI_SUELD_OTA",
        "AJU_II_PP_AP_AAS_PP",
        "tot_empleo",
        "admin_per_dir",
        "admin_tem_dir",
        "admin_per_emp",
        "admin_per_apr",
    ]

    company_prod_per = company_averange_salary.copy()
    company_prod_per = company_prod_per.drop(columns=colums_to_delete_prd, axis=1)
    company_prod_per = company_prod_per.rename(
        columns={
            "II_PP_PP__NPERS_OP": "#_prod_dir",
            "II_PP_TD__NPERS_OT": "#_prod_tem_dir",
            "II_PP_TI__NPERS_OTA": "#_prod_tem_emp",
            "II_PP_AP__APEP": "#_prod_apr",
        }
    )
    prod_per_order = [
        "#_prod_dir",
        "pro_per_dir",
        "#_prod_tem_dir",
        "pro_tem_dir",
        "#_prod_tem_emp",
        "pro_per_emp",
        "#_prod_apr",
        "pro_per_apr",
    ]
    company_prod_per = company_prod_per.reindex(columns=prod_per_order)

    return company_prod_per, company_admin_per


company_per_pro, company_per_adm = all_personal(company_data, company_nordem)
company_per_adm
# company_per_pro

#_admin_per_dir  admin_per_dir  #_admin_tem_dir  admin_tem_dir  \
anio mes                                                                   
2021 7                677          5,502               67          2,192   
     8                679          5,579               77          2,049   
     9                697          5,500               94          1,737   
     10               719          5,604               98          1,604   
     11               717          5,672              103          1,578   
     12               723          5,536              118          1,576   
2022 1                722          5,051              121          1,504   
     2                722          6,180              120          1,567   
     3                728          6,109              112          1,523   
     4                724          6,398              108          1,633   
     5                727          6,400              104          1,610   
     6                731          6,528              101          1,696   
     7                740          6,512               94          1,695   
     8                735          6,604               97          1,853   
     9                748          6,807               92          1,879   
     10               754          6,382               92          1,931   
     11               760          6,348               85          2,017   
     12               753          7,116               81          2,040   
2023 1                746          5,504               79          1,907   
     2                736          6,891               77          1,984   
     3                727          6,825               76          1,963   
     4                723          7,172               73          2,121   
     5                730          7,200               72          2,211   
     6                722          7,143               67          2,190   
     7                713          7,081               58          2,270   

          #_admin_tem_emp  admin_per_emp  #_admin_ape  admin_per_apr  
anio mes                                                              
2021 7                 13          3,124          109            906  
     8                 15          3,139          106            907  
     9                 14          3,522           98            911  
     10                 9          3,522          108            908  
     11                11          3,385          108            910  
     12                14          3,177          115             91  
2022 1                 14          2,779           95            845  
     2                 10          4,102          100            890  
     3                  6          4,742           98            919  
     4                 11          3,459          119            796  
     5                  7          3,658          118            816  
     6                  5          3,516           97            840  
     7                  5          4,509           87          1,107  
     8                  5          3,648           91            891  
     9                  5          3,758          103            905  
     10                 9          3,535          102            920  
     11                 9          3,784          105            894  
     12                 9          3,785          113            828  
2023 1                  8          3,981          104            949  
     2                  7          4,085          105            945  
     3                  8          5,580          113            931  
     4                 19          6,093          128            938  
     5                 19          5,113          118            949  
     6                 22          5,446          115            972  
     7                 14          5,991          128            962

In [54]:
production_data = company_data.copy()
production_data = production_data[production_data["id_numord"] == company_nordem]
production_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 5616
Data columns (total 81 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        25 non-null     int64  
 1   Llave                             25 non-null     int64  
 2   anio                              25 non-null     int64  
 3   mes                               25 non-null     int64  
 4   novedad                           25 non-null     int64  
 5   id_estado                         25 non-null     int64  
 6   imp_Imputacion                    24 non-null     object 
 7   Corrección                        24 non-null     object 
 8   id_numord                         25 non-null     int64  
 9   nombre_estab                      25 non-null     object 
 10  Dominio_39                        25 non-null     int64  
 11  nombreDpto                        25 non-null     object 
 12  nombreMp

In [55]:
def production(company_data, company_nordem):
    production_data = company_data.copy()
    production_data = production_data[production_data["id_numord"] == company_nordem]

    # Sumatory of all salaries
    production_data["sum_salarios"] = (
        production_data["AJU_II_PA_PP_SUELD_EP"]
        + production_data["AJU_II_PA_TD_SUELD_ET"]
        + production_data["AJU_II_PA_TI_SUELD_ETA"]
        + production_data["AJU_II_PA_AP_AAS_AP"]
        + production_data["AJU_II_PP_PP_SUELD_OP"]
        + production_data["AJU_II_PP_TD_SUELD_OT"]
        + production_data["AJU_II_PP_TI_SUELD_OTA"]
        + production_data["AJU_II_PP_AP_AAS_PP"]
    )

    # operation of averange salary for perosnal
    production_data["sal_percapita"] = (
        production_data["sum_salarios"] / production_data["tot_empleo"]
    )

    # diference of production and sells
    production_data["pro-ven"] = (
        production_data["AJU_III_PE_PRODUCCION"] - production_data["total_ventas"]
    )

    # diference of inventory
    # production_data["previus_onvent"] =
    production_data["dif-inv"] = production_data["III_EX__VEXIS"] - production_data[
        "III_EX__VEXIS"
    ].shift(periods=1, fill_value=None)

    # inventory coeficient

    production_data["coeficiente"] = production_data.apply(
        lambda row: row["pro-ven"] / row["dif-inv"] if row["dif-inv"] != 0 else 0,
        axis=1,
    )
    print(production_data["coeficiente"])

    production_data = production_data.pivot_table(
        index=["anio", "mes"],
        values=[
            "AJU_III_PE_PRODUCCION",
            "AJU_III_PE_VENTASIN",
            "AJU_III_PE_VENTASEX",
            "total_ventas",
            "III_EX__VEXIS",
            "sal_percapita",
            "tot_operativo",
            "tot_empleo",
            "dif-inv",
            "pro-ven",
            "coeficiente",
            "AJU_II_HORAS_HORDI_T",
            "AJU_II_HORAS_HEXTR_T",
        ],
    )

    # aplay fortmat for presentation of the table.
    # i use a lambda funtion, for each x put this format

    production_data["coeficiente"] = production_data["coeficiente"].apply(
        lambda x: "{:.2f}".format(x)
    )

    # Change name of colums
    production_data = production_data.rename(
        columns={
            "AJU_III_PE_PRODUCCION": "Produccion",
            "AJU_III_PE_VENTASIN": "Ventas_int",
            "AJU_III_PE_VENTASEX": "Ventas_ext",
            "III_EX__VEXIS": "Existen",
            "AJU_II_HORAS_HORDI_T": "Horas_Ord",
            "AJU_II_HORAS_HEXTR_T": "Horas_Ext",
        }
    )

    # reset the index of table
    production_data = production_data.reset_index()
    production_data = production_data.copy()

    production_data["Periodo"] = pd.to_datetime(
        production_data["anio"].astype(str) + "-" + production_data["mes"].astype(str),
        format="%Y-%m",
    )

    # Order columns
    prod_order = [
        "Periodo",
        "Produccion",
        "Ventas_int",
        "Ventas_ext",
        "total_ventas",
        "Existen",
        "tot_empleo",
        "tot_operativo",
        "sal_percapita",
        "pro-ven",
        "dif-inv",
        "coeficiente",
        "Horas_Ord",
        "Horas_Ext",
    ]

    production_data = production_data.reindex(columns=prod_order)

    return production_data


producion_of_company = production(company_data, company_nordem)
producion_of_company

0      NaN
234      1
468      1
702      1
936      0
1170     1
1404     1
1638    -4
1872     1
2106     5
2340     1
2574     1
2808     1
3042     1
3276     1
3510     1
3744     2
3978     1
4212     8
4446     1
4680     1
4914     1
5148     2
5382     1
5616     1
Name: coeficiente, dtype: float64


,Periodo,Produccion,Ventas_int,Ventas_ext,total_ventas,Existen,tot_empleo,tot_operativo,sal_percapita,pro-ven,dif-inv,coeficiente,Horas_Ord,Horas_Ext
0,2021-07-01,"84,600,363","58,500,550","28,137,699","86,638,249","28,993,245","2,307","1,441","3,574","-2,037,886",NaN,nan,"253,631","23,781"
1,2021-08-01,"99,705,399","62,021,973","34,517,004","96,538,977","31,440,081","2,328","1,451","3,594","3,166,422","2,446,836",1.29,"255,274","23,248"
2,2021-09-01,"77,290,325","58,373,997","15,990,227","74,364,224","33,699,959","2,375","1,472","3,606","2,926,101","2,259,878",1.29,"259,029","30,724"
3,2021-10-01,"84,231,642","60,032,334","23,371,888","83,404,222","34,336,436","2,392","1,458","3,544","827,420","636,477",1.30,"256,673","23,768"
4,2021-11-01,"81,154,372","61,107,283","20,047,089","81,154,372","34,866,544","2,495","1,556","3,590",0,"530,108",0.00,"293,040","37,009"
5,2021-12-01,"140,811,693","93,860,824","44,650,868","138,511,692","36,750,338","2,545","1,575","3,501","2,300,001","1,883,794",1.22,"277,043","29,594"
6,2022-01-01,"86,655,430","44,017,265","27,195,092","71,212,357","48,699,163","2,533","1,581","3,148","15,443,073","11,948,825",1.29,"252,881","26,953"
7,2022-02-01,"65,738,200","51,271,034","21,336,361","72,607,395","50,576,438","2,475","1,523","3,774","-6,869,195","1,877,275",-3.66,"263,654","21,909"
8,2022-03-01,"76,705,615","52,651,162","19,007,490","71,658,652","54,460,472","2,409","1,465","3,762","5,046,963","3,884,034",1.30,"257,549","69,549"
9,2022-04-01,"101,530,000","68,836,475","34,483,119","103,319,594","54,076,479","2,447","1,485","3,888","-1,789,594","-383,993",4.66,"235,213","100,481"


In [56]:
producion_of_company.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Periodo        25 non-null     datetime64[ns]
 1   Produccion     25 non-null     float64       
 2   Ventas_int     25 non-null     float64       
 3   Ventas_ext     25 non-null     float64       
 4   total_ventas   25 non-null     float64       
 5   Existen        25 non-null     float64       
 6   tot_empleo     25 non-null     float64       
 7   tot_operativo  25 non-null     float64       
 8   sal_percapita  25 non-null     float64       
 9   pro-ven        25 non-null     float64       
 10  dif-inv        24 non-null     float64       
 11  coeficiente    25 non-null     object        
 12  Horas_Ord      25 non-null     float64       
 13  Horas_Ext      25 non-null     float64       
dtypes: datetime64[ns](1), float64(12), object(1)
memory usage: 2.9+ KB


In [57]:
# with use of groupby i generate the total per year
resume_table = producion_of_company.groupby(
    producion_of_company["Periodo"].dt.year
).agg({"Produccion": np.sum, "total_ventas": np.sum})
resume_table = resume_table.reset_index(drop=False)
resume_table

,Periodo,Produccion,total_ventas
0,2021,"567,793,794","560,611,736"
1,2022,"1,157,819,948","1,111,468,226"
2,2023,"599,093,696","653,313,773"


In [58]:
#! resume to persoanl hours

for_drop = [
    "Produccion",
    "Ventas_int",
    "Ventas_ext",
    "total_ventas",
    "Existen",
    "tot_empleo",
    "sal_percapita",
    "pro-ven",
    "dif-inv",
    "coeficiente",
]
final_mount = producion_of_company.copy()
final_mount = final_mount.drop(columns=for_drop)
before_period = final_mount.iloc[-12]
before_period = before_period.to_frame().T
final_mount = final_mount.iloc[-2:]
final_mount = pd.concat([before_period, final_mount], axis=0)
final_mount["promedio_30"] = (
    (final_mount["Horas_Ord"] + final_mount["Horas_Ext"]) / 30
) / final_mount["tot_operativo"]
final_mount["promedio_29"] = (
    (final_mount["Horas_Ord"] + final_mount["Horas_Ext"]) / 29
) / final_mount["tot_operativo"]
final_mount

,Periodo,tot_operativo,Horas_Ord,Horas_Ext,promedio_30,promedio_29
13,2022-08-01,"1,644","276,123","107,837",8,8
23,2023-06-01,"1,543","283,857","95,094",8,8
24,2023-07-01,"1,498","275,649","89,768",8,8


In [95]:
# Variation of mount and year personal. production
#  production variation
def production_var(data):
    actual_mount = data.iloc[-1]
    actual_mount = actual_mount.to_frame().T
    before_mount = data.iloc[-2]
    before_mount = before_mount.to_frame().T

    colums = [
        "Produccion",
        "Ventas_int",
        "Ventas_ext",
        "total_ventas",
        "tot_empleo",
        "tot_operativo",
        "sal_percapita",
        "Horas_Ord",
        "Horas_Ext",
    ]

    mount = pd.concat([before_mount, actual_mount])
    mount = mount.drop(
        columns=[
            "Existen",
            "pro-ven",
            "dif-inv",
            "coeficiente",
        ]
    )
    variation = mount[colums].pct_change() * 100
    mount = pd.concat([mount, variation])
    mount = mount.reset_index(drop=False)
    mount = mount.rename(index={mount.index[-1]: "Variacion"})
    mount = mount.drop(2)
    mount = mount.drop(columns="index")
    mount = mount.reset_index(drop=False)

    twuelve_mount = data.iloc[-13]
    twuelve_mount = twuelve_mount.to_frame().T
    year = pd.concat([twuelve_mount, actual_mount])
    year = year.drop(
        columns=[
            "Existen",
            "pro-ven",
            "dif-inv",
            "coeficiente",
        ]
    )
    variation_year = year[colums].pct_change() * 100
    year = pd.concat([year, variation_year])
    year = year.reset_index(drop=False)
    year = year.rename(index={year.index[-1]: "Variacion"})
    year = year.drop(2)
    year = year.drop(columns="index")
    year = year.reset_index(drop=False)

    return mount, year


mesual, year = production_var(producion_of_company)
mesual

,index,Periodo,Produccion,Ventas_int,Ventas_ext,total_ventas,tot_empleo,tot_operativo,sal_percapita,Horas_Ord,Horas_Ext
0,0,2023-06-01,"103,130,890","76,152,043","27,961,543","104,113,586","2,469","1,543","4,415","283,857","95,094"
1,1,2023-07-01,"73,800,900","65,070,431","27,174,782","92,245,213","2,411","1,498","4,371","275,649","89,768"
2,Variacion,NaT,-28,-15,-3,-11,-2,-3,-1,-3,-6


In [96]:
year

,index,Periodo,Produccion,Ventas_int,Ventas_ext,total_ventas,tot_empleo,tot_operativo,sal_percapita,Horas_Ord,Horas_Ext
0,0,2022-07-01,"107,950,281","62,109,130","43,129,496","105,238,626","2,522","1,596","3,976","268,096","73,914"
1,1,2023-07-01,"73,800,900","65,070,431","27,174,782","92,245,213","2,411","1,498","4,371","275,649","89,768"
2,Variacion,NaT,-32,5,-37,-12,-4,-6,10,3,21


In [115]:
def var_personal(data):
    actual_mount = data.iloc[-1]
    actual_mount = actual_mount.to_frame().T
    mount_before = data.iloc[-2]
    mount_before = mount_before.to_frame().T
    mount = pd.concat([mount_before, actual_mount])
    variation = mount.pct_change() * 100
    mount = pd.concat([mount, variation])
    mount = mount.reset_index(drop=False)
    mount = mount.rename(index={mount.index[-1]: "Variacion"})
    mount = mount.drop(2)

    before_year = data.iloc[-13]
    before_year = before_year.to_frame().T
    year = pd.concat([before_year, actual_mount])
    variation_year = year.pct_change() * 100
    year = pd.concat([year, variation_year])
    year = year.reset_index(drop=False)
    year = year.rename(index={year.index[-1]: "Variacion"})
    year = year.drop(2)

    return mount, year


mes_var_per, año_var_per = var_personal(company_per_pro)
mes_var_per

,level_0,level_1,#_prod_dir,pro_per_dir,#_prod_tem_dir,pro_tem_dir,#_prod_tem_emp,pro_per_emp,#_prod_apr,pro_per_apr
0,2023,6,"1,187","3,708",221,"1,905",135,"4,016",0,NaN
1,2023,7,"1,161","3,672",237,"1,913",100,"4,345",0,NaN
Variacion,2023,7,-2,-1,7,0,-26,8,NaN,NaN


In [116]:
año_var_per

,level_0,level_1,#_prod_dir,pro_per_dir,#_prod_tem_dir,pro_tem_dir,#_prod_tem_emp,pro_per_emp,#_prod_apr,pro_per_apr
0,2022,7,"1,203","3,257",200,"1,676",193,"3,508",0,NaN
1,2023,7,"1,161","3,672",237,"1,913",100,"4,345",0,NaN
Variacion,2023,7,-3,13,19,14,-48,24,NaN,NaN
